# Modelling

7. Modelling <br>

8. Conclusion and Recommendations

# 7. Modelling

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

#Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

#Adjust the max columns displayed in the df
pd.set_option("display.max_columns", 80)
pd.set_option("display.max_rows", 80)

#Load csv file
df_train_model = pd.read_csv("../Dataset/train_model.csv")

To carry out One Hot Encoding (OHE) of nominal variables and mapping of ordinal variables. 

In [2]:
# Use OneHotEncoder
# instantiate the OHE
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
# fit OHE to features
ohe.fit(df_train_model[['neighborhood', 'exterior_1st', 'exterior_2nd', 'foundation']])
# create dummy array
transformed = ohe.transform(df_train_model[['neighborhood', 'exterior_1st', 'exterior_2nd', 'foundation']])
# input column names
column_name = ohe.get_feature_names_out(['neighborhood', 'exterior_1st', 'exterior_2nd', 'foundation'])
# rebuild dataframe and drop redundant columns
df_train_model = pd.concat([df_train_model, pd.DataFrame(transformed, columns=column_name)], axis=1)
df_train_model.drop(['neighborhood', 'exterior_1st', 'exterior_2nd', 'foundation'], axis=1, inplace=True)
df_train_model.head()

,overall_qual,log_gr_liv_area,garage_area,total_bsmt_sf,exter_qual,bsmt_qual,bsmt_exposure,bsmtfin_type_1,heating_qc,kitchen_qual,fireplace_qu,garage_finish,saleprice,neighborhood_Blmngtn,neighborhood_Blueste,neighborhood_BrDale,neighborhood_BrkSide,neighborhood_ClearCr,neighborhood_CollgCr,neighborhood_Crawfor,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Greens,neighborhood_GrnHill,neighborhood_IDOTRR,neighborhood_Landmrk,neighborhood_MeadowV,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NPkVill,neighborhood_NWAmes,neighborhood_NoRidge,neighborhood_NridgHt,neighborhood_OldTown,neighborhood_SWISU,neighborhood_Sawyer,neighborhood_SawyerW,neighborhood_Somerst,neighborhood_StoneBr,neighborhood_Timber,neighborhood_Veenker,exterior_1st_AsbShng,exterior_1st_AsphShn,exterior_1st_BrkComm,exterior_1st_BrkFace,exterior_1st_CBlock,exterior_1st_CemntBd,exterior_1st_HdBoard,exterior_1st_ImStucc,exterior_1st_MetalSd,exterior_1st_Plywood,exterior_1st_Stone,exterior_1st_Stucco,exterior_1st_VinylSd,exterior_1st_Wd Sdng,exterior_1st_WdShing,exterior_2nd_AsbShng,exterior_2nd_AsphShn,exterior_2nd_Brk Cmn,exterior_2nd_BrkFace,exterior_2nd_CBlock,exterior_2nd_CmentBd,exterior_2nd_HdBoard,exterior_2nd_ImStucc,exterior_2nd_MetalSd,exterior_2nd_Plywood,exterior_2nd_Stone,exterior_2nd_Stucco,exterior_2nd_VinylSd,exterior_2nd_Wd Sdng,exterior_2nd_Wd Shng,foundation_BrkTil,foundation_CBlock,foundation_PConc,foundation_Slab,foundation_Stone,foundation_Wood
0,6,7.299121,475.0,725.0,Gd,TA,No,GLQ,Ex,Gd,No Fireplace,RFn,130500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,7,7.660114,559.0,913.0,Gd,Gd,No,GLQ,Ex,Gd,TA,RFn,220000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,5,6.963190,246.0,1057.0,TA,TA,No,GLQ,TA,Gd,No Fireplace,Unf,109000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,5,7.275172,400.0,384.0,TA,Gd,No,Unf,Gd,TA,No Fireplace,Fin,174000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,6,7.275865,484.0,676.0,TA,Fa,No,Unf,TA,TA,No Fireplace,Unf,138500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Mapping of ordinal variables

In [3]:
#mapping

exter_qual_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4}
bsmt_qual_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'No Basement':6}
bsmt_exposure_map = {'Gd':1, 'Av':2, 'Mn':3, 'No':4}
bsmtfin_type_1_map = {'GLQ': 1, 'ALQ': 2, 'BLQ': 3, 'Rec': 4, 'LwQ': 5, 'Unf': 6, 'No Basement': 7}
heating_qc_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po': 5}
kitchen_qual_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po': 5}
fireplace_qu_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'No Fireplace':6}
garage_finish_map = {'Fin':1, 'RFn':2, 'Unf':3, 'No Garage':4}

df_train_model["exter_qual"] = df_train_model["exter_qual"].map(exter_qual_map)
df_train_model["bsmt_qual"] = df_train_model["bsmt_qual"].map(bsmt_qual_map)
df_train_model["bsmt_exposure"] = df_train_model["bsmt_exposure"].map(bsmt_exposure_map)
df_train_model["bsmtfin_type_1"] = df_train_model["bsmtfin_type_1"].map(bsmtfin_type_1_map)
df_train_model["heating_qc"] = df_train_model["heating_qc"].map(heating_qc_map)
df_train_model["kitchen_qual"] = df_train_model["kitchen_qual"].map(kitchen_qual_map)
df_train_model["fireplace_qu"] = df_train_model["fireplace_qu"].map(fireplace_qu_map)
df_train_model["garage_finish"] = df_train_model["garage_finish"].map(garage_finish_map)

df_train_model.head()

,overall_qual,log_gr_liv_area,garage_area,total_bsmt_sf,exter_qual,bsmt_qual,bsmt_exposure,bsmtfin_type_1,heating_qc,kitchen_qual,fireplace_qu,garage_finish,saleprice,neighborhood_Blmngtn,neighborhood_Blueste,neighborhood_BrDale,neighborhood_BrkSide,neighborhood_ClearCr,neighborhood_CollgCr,neighborhood_Crawfor,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Greens,neighborhood_GrnHill,neighborhood_IDOTRR,neighborhood_Landmrk,neighborhood_MeadowV,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NPkVill,neighborhood_NWAmes,neighborhood_NoRidge,neighborhood_NridgHt,neighborhood_OldTown,neighborhood_SWISU,neighborhood_Sawyer,neighborhood_SawyerW,neighborhood_Somerst,neighborhood_StoneBr,neighborhood_Timber,neighborhood_Veenker,exterior_1st_AsbShng,exterior_1st_AsphShn,exterior_1st_BrkComm,exterior_1st_BrkFace,exterior_1st_CBlock,exterior_1st_CemntBd,exterior_1st_HdBoard,exterior_1st_ImStucc,exterior_1st_MetalSd,exterior_1st_Plywood,exterior_1st_Stone,exterior_1st_Stucco,exterior_1st_VinylSd,exterior_1st_Wd Sdng,exterior_1st_WdShing,exterior_2nd_AsbShng,exterior_2nd_AsphShn,exterior_2nd_Brk Cmn,exterior_2nd_BrkFace,exterior_2nd_CBlock,exterior_2nd_CmentBd,exterior_2nd_HdBoard,exterior_2nd_ImStucc,exterior_2nd_MetalSd,exterior_2nd_Plywood,exterior_2nd_Stone,exterior_2nd_Stucco,exterior_2nd_VinylSd,exterior_2nd_Wd Sdng,exterior_2nd_Wd Shng,foundation_BrkTil,foundation_CBlock,foundation_PConc,foundation_Slab,foundation_Stone,foundation_Wood
0,6,7.299121,475.0,725.0,2,3,4,1,1,2,6,2,130500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,7,7.660114,559.0,913.0,2,2,4,1,1,2,3,2,220000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,5,6.963190,246.0,1057.0,3,3,4,1,3,2,6,3,109000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,5,7.275172,400.0,384.0,3,2,4,6,2,3,6,1,174000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,6,7.275865,484.0,676.0,3,4,4,6,3,3,6,3,138500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Baseline Model

Build our baseline model on the null model and find the R2 score and RMSE value.

In [4]:
#define our X and Y variables

X = df_train_model.drop(['saleprice'], axis=1)
y = df_train_model['saleprice']

R2 score and RMSE value of baseline model

In [5]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X, y)
y_pred = dummy_regr.predict(X)

R2 score for null model is zero as the proposed model is the mean model.

In [6]:
#R2 score

dummy_regr.score(X, y)

0.0

RMSE value

In [7]:
#RMSE using sklearn

metrics.mean_squared_error(y, y_pred, squared = False)

79292.58987577505

In [8]:
#RMSE manual calculation matches value of sklearn

y_bar = np.mean(y)
null_mse = np.mean((y - y_bar)**2)
null_mse**0.5

79292.58987577507

## Model Evaluation Criteria

The following Linear Regression and Regularization models should aim to achieve a lower RMSE value and higher R2 score than the null model.

1. R2 score: R2 is the proportional improvement in prediction from the regression model, compared to the mean model. A higher R2 score would mean that the regression model is better at predicting the value of our target, which is saleprice in this case.<br>
2. RMSE value: square root of the average squared difference between difference our prediction vs. the actual value of the target. A lower RMSE value would mean that the difference between the prediction and actual value of saleprice is small and a better prediction of our target, which is saleprice.

## Linear Regression Model

Build linear regression model

In [9]:
#run train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

R2 score for train set with cross validation

In [11]:
#R2 score

cross_val_score(lr, X_train, y_train, cv=5).mean()

0.8266500399560082

R2 score on test set

In [12]:
#R2 score

r2_score(y_test, lr_pred)

0.8329695404329658

Based on the R2 score for train set and test set, the difference is <1%. This indicates that there is a balance between high bias and high variance and there is no underfitting or overfitting of our model.

MSE

In [13]:
#MSE

metrics.mean_squared_error(y_test, lr_pred)

1079676314.920952

RMSE

In [14]:
#RMSE

metrics.mean_squared_error(y_test, lr_pred, squared = False)

32858.42836961245

We can see here that the RMSE value of Linear Regression model is less than half of our baseline model and it has a better R2 score. We want a lower RMSE value and higher R2 score. Hence, the Linear Regression Model performs better than our baseline model.

## Regularization Models

Since we are using Lasso and Ridge, need to scale our features.

In [15]:
ss = StandardScaler()
ss.fit(X_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

## Ridge Regression (L2) Model

Build our Ridge Regression Model

In [16]:
ridgeRegressor = Ridge()
ridgeRegressor.fit(X_train_sc, y_train)
ridge_pred = ridgeRegressor.predict(X_test_sc)

R2 score for train set with cross validation

In [17]:
cross_val_score(ridgeRegressor, X_train, y_train, cv=5).mean()

0.8293437167655849

R2 score on test set

In [18]:
#R2 score

r2_score(y_test, ridge_pred)

0.83300875850344

Based on the R2 score for train set and test set, the difference is ~1%. This indicates that there is a balance between high bias and high variance and there is no underfitting or overfitting of our model.

MSE

In [19]:
#MSE

metrics.mean_squared_error(y_test, ridge_pred)

1079422811.3269508

RMSE

In [20]:
#RMSE

metrics.mean_squared_error(y_test, ridge_pred, squared = False)

32854.57063068928

Ridge Regression Model performs better than our baseline model based on R2 score and RMSE value. R2 scores and RMSE value of Ridge Regression Model fairs slightly better than Linear Regression Model.

## Lasso Regression (L1) Model

Build our Ridge Regression Model

In [21]:
lassoRegressor = Lasso()
lassoRegressor.fit(X_train_sc, y_train)
lasso_pred = lassoRegressor.predict(X_test_sc)

R2 score for train set with cross validation

In [22]:
cross_val_score(lassoRegressor, X_train, y_train, cv=5).mean()

0.8270682994878585

R2 score on test set

In [23]:
#R2 score

r2_score(y_test, lasso_pred)

0.832974575357788

Based on the R2 score for train set and test set, the difference is <1%. This indicates that there is a balance between high bias and high variance and there is no underfitting or overfitting of our model.

MSE

In [24]:
#MSE

metrics.mean_squared_error(y_test, lasso_pred)

1079643769.4254065

RMSE

In [25]:
#RMSE

metrics.mean_squared_error(y_test, lasso_pred, squared = False)

32857.93312771524

Lasso Regression Model performs better than our baseline model based on R2 score and RMSE value. R2 scores and RMSE value of Lasso Regression Model fairs slightly better than Linear Regression Model.

## Chosen Model to predict saleprice of properties in Ames, Iowa

With the above analysis, we will use the Ridge Regression model to predict saleprice in Ames, Iowa given that the R2 score is the highest and RMSE value is lowest of all the models analysed.

## Taking a look at the lasso coefficient of features applied to our Regression Model

Features with a higher Lasso coefficient value includes gr_liv_area, overall_qual, garage_area and neighbourhood which is not unexpected based on our analysis during EDA.

In [26]:
# Create df for storing coefficients and their corresponding variable names
coefficient_df = pd.DataFrame()

# Storing all the column names of predictors
coefficient_df['Column_Name'] = X_train.columns

# Store coefficients of predictors and sort according to the coefficient value
coefficient_df['Coefficient_Value'] = pd.Series(lassoRegressor.coef_)
coefficient_df.sort_values(by = 'Coefficient_Value', inplace = True, ascending = False)

# Preview variables' coefficient values
# The higher the values, the stronger the variables
coefficient_df.head(10)

,Column_Name,Coefficient_Value
1,log_gr_liv_area,20783.627569
45,exterior_1st_CemntBd,18808.749880
0,overall_qual,13613.117364
2,garage_area,8527.087604
31,neighborhood_NridgHt,8031.761491
37,neighborhood_StoneBr,7536.221002
3,total_bsmt_sf,7497.478684
30,neighborhood_NoRidge,7033.559824
73,foundation_Slab,3690.099388
18,neighborhood_Crawfor,3141.037014


## Data Cleaning of Test file for Kaggle Submission

In [27]:
#load test csv file

df_test = pd.read_csv("../Dataset/test.csv")

Filter the features required in the model + Id and save under new df

In [28]:
df_test_clean = df_test.copy().filter(['Id', 'Overall Qual', 'Gr Liv Area', 'Garage Area', 'Total Bsmt SF', 'Neighborhood', 'Exterior 1st', 'Exterior 2nd', 'Exter Qual', 'Foundation', 'Bsmt Qual', 'Bsmt Exposure', 'BsmtFin Type 1', 'Heating QC', 'Kitchen Qual', 'Fireplace Qu', 'Garage Finish'], axis=1)

Take a look at the data

In [29]:
df_test_clean.head()

,Id,Overall Qual,Gr Liv Area,Garage Area,Total Bsmt SF,Neighborhood,Exterior 1st,Exterior 2nd,Exter Qual,Foundation,Bsmt Qual,Bsmt Exposure,BsmtFin Type 1,Heating QC,Kitchen Qual,Fireplace Qu,Garage Finish
0,2658,6,1928,440,1020,OldTown,AsbShng,AsbShng,TA,Stone,Fa,No,Unf,Gd,Fa,NaN,Unf
1,2718,5,1967,580,1967,Sawyer,Plywood,Plywood,TA,CBlock,Gd,No,Unf,TA,TA,NaN,Fin
2,2414,7,1496,426,654,Gilbert,VinylSd,VinylSd,Gd,PConc,Gd,Av,GLQ,Ex,Gd,Gd,RFn
3,1989,5,968,480,968,OldTown,Wd Sdng,Wd Sdng,Gd,CBlock,TA,No,Unf,TA,TA,NaN,Unf
4,625,6,1394,514,1394,NAmes,Plywood,Plywood,TA,CBlock,Gd,No,BLQ,Gd,TA,Gd,RFn


Address the null values

In [30]:
df_test_clean.isnull().sum()

Id                  0
Overall Qual        0
Gr Liv Area         0
Garage Area         0
Total Bsmt SF       0
Neighborhood        0
Exterior 1st        0
Exterior 2nd        0
Exter Qual          0
Foundation          0
Bsmt Qual          25
Bsmt Exposure      25
BsmtFin Type 1     25
Heating QC          0
Kitchen Qual        0
Fireplace Qu      422
Garage Finish      45
dtype: int64

Imputation of null values

In [31]:
df_test_clean['Bsmt Qual'] = df_test_clean['Bsmt Qual'].fillna("No Basement")
df_test_clean['Bsmt Exposure'] = df_test_clean['Bsmt Exposure'].fillna("No")
df_test_clean['BsmtFin Type 1'] = df_test_clean['BsmtFin Type 1'].fillna("No Basement")
df_test_clean['Fireplace Qu'] = df_test_clean['Fireplace Qu'].fillna("No Fireplace")
df_test_clean['Garage Finish'] = df_test_clean['Garage Finish'].fillna("No Garage")

All null values dealt with

In [32]:
df_test_clean.isnull().sum()

Id                0
Overall Qual      0
Gr Liv Area       0
Garage Area       0
Total Bsmt SF     0
Neighborhood      0
Exterior 1st      0
Exterior 2nd      0
Exter Qual        0
Foundation        0
Bsmt Qual         0
Bsmt Exposure     0
BsmtFin Type 1    0
Heating QC        0
Kitchen Qual      0
Fireplace Qu      0
Garage Finish     0
dtype: int64

Take a look at all entries and columns

In [33]:
df_test_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Id              878 non-null    int64 
 1   Overall Qual    878 non-null    int64 
 2   Gr Liv Area     878 non-null    int64 
 3   Garage Area     878 non-null    int64 
 4   Total Bsmt SF   878 non-null    int64 
 5   Neighborhood    878 non-null    object
 6   Exterior 1st    878 non-null    object
 7   Exterior 2nd    878 non-null    object
 8   Exter Qual      878 non-null    object
 9   Foundation      878 non-null    object
 10  Bsmt Qual       878 non-null    object
 11  Bsmt Exposure   878 non-null    object
 12  BsmtFin Type 1  878 non-null    object
 13  Heating QC      878 non-null    object
 14  Kitchen Qual    878 non-null    object
 15  Fireplace Qu    878 non-null    object
 16  Garage Finish   878 non-null    object
dtypes: int64(5), object(12)
memory usage: 116.7+ KB


To change dtype of Garage Area and Total Bsmt SF to float

In [34]:
df_test_clean = df_test_clean.astype({'Garage Area': 'float64', 'Total Bsmt SF': 'float64'})
df_test_clean.dtypes

Id                  int64
Overall Qual        int64
Gr Liv Area         int64
Garage Area       float64
Total Bsmt SF     float64
Neighborhood       object
Exterior 1st       object
Exterior 2nd       object
Exter Qual         object
Foundation         object
Bsmt Qual          object
Bsmt Exposure      object
BsmtFin Type 1     object
Heating QC         object
Kitchen Qual       object
Fireplace Qu       object
Garage Finish      object
dtype: object

Rename columns

In [35]:
df_test_clean.rename(columns={
    'Id': 'id',
    'Overall Qual': 'overall_qual',
    'Gr Liv Area': 'gr_liv_area',
    'Garage Area': 'garage_area',
    'Total Bsmt SF': 'total_bsmt_sf',
    'Neighborhood': 'neighborhood',
    'Exterior 1st': 'exterior_1st',
    'Exterior 2nd': 'exterior_2nd',
    'Exter Qual': 'exter_qual',
    'Foundation': 'foundation',
    'Bsmt Qual': 'bsmt_qual',
    'Bsmt Exposure': 'bsmt_exposure',
    'BsmtFin Type 1': 'bsmtfin_type_1',
    'Heating QC': 'heating_qc',
    'Kitchen Qual': 'kitchen_qual',
    'Fireplace Qu': 'fireplace_qu',
    'Garage Finish': 'garage_finish'
}, inplace=True)

log the gr_liv_area column

In [36]:
#log the gr_liv_area column

df_test_clean['gr_liv_area'] = np.log(df_test_clean['gr_liv_area'])

#rename to log_gr_liv_area

df_test_clean.rename(columns={'gr_liv_area': 'log_gr_liv_area'}, inplace=True)

OHE + Mapping of categorial variables

In [37]:
#create a new df to do OHE + Mapping

df_test_model = df_test_clean.copy()

In [38]:
#OHE

#create dummy array
df_test_transform = ohe.transform(df_test_model[['neighborhood', 'exterior_1st', 'exterior_2nd', 'foundation']])
# input column names
column_name = ohe.get_feature_names_out(['neighborhood', 'exterior_1st', 'exterior_2nd', 'foundation'])
# rebuild dataframe and drop redundant columns
df_test_model = pd.concat([df_test_model, pd.DataFrame(df_test_transform, columns=column_name)], axis=1)
df_test_model.drop(['neighborhood', 'exterior_1st', 'exterior_2nd', 'foundation'], axis=1, inplace=True)
df_test_model.head()

,id,overall_qual,log_gr_liv_area,garage_area,total_bsmt_sf,exter_qual,bsmt_qual,bsmt_exposure,bsmtfin_type_1,heating_qc,kitchen_qual,fireplace_qu,garage_finish,neighborhood_Blmngtn,neighborhood_Blueste,neighborhood_BrDale,neighborhood_BrkSide,neighborhood_ClearCr,neighborhood_CollgCr,neighborhood_Crawfor,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Greens,neighborhood_GrnHill,neighborhood_IDOTRR,neighborhood_Landmrk,neighborhood_MeadowV,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NPkVill,neighborhood_NWAmes,neighborhood_NoRidge,neighborhood_NridgHt,neighborhood_OldTown,neighborhood_SWISU,neighborhood_Sawyer,neighborhood_SawyerW,neighborhood_Somerst,neighborhood_StoneBr,neighborhood_Timber,neighborhood_Veenker,exterior_1st_AsbShng,exterior_1st_AsphShn,exterior_1st_BrkComm,exterior_1st_BrkFace,exterior_1st_CBlock,exterior_1st_CemntBd,exterior_1st_HdBoard,exterior_1st_ImStucc,exterior_1st_MetalSd,exterior_1st_Plywood,exterior_1st_Stone,exterior_1st_Stucco,exterior_1st_VinylSd,exterior_1st_Wd Sdng,exterior_1st_WdShing,exterior_2nd_AsbShng,exterior_2nd_AsphShn,exterior_2nd_Brk Cmn,exterior_2nd_BrkFace,exterior_2nd_CBlock,exterior_2nd_CmentBd,exterior_2nd_HdBoard,exterior_2nd_ImStucc,exterior_2nd_MetalSd,exterior_2nd_Plywood,exterior_2nd_Stone,exterior_2nd_Stucco,exterior_2nd_VinylSd,exterior_2nd_Wd Sdng,exterior_2nd_Wd Shng,foundation_BrkTil,foundation_CBlock,foundation_PConc,foundation_Slab,foundation_Stone,foundation_Wood
0,2658,6,7.564238,440.0,1020.0,TA,Fa,No,Unf,Gd,Fa,No Fireplace,Unf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2718,5,7.584265,580.0,1967.0,TA,Gd,No,Unf,TA,TA,No Fireplace,Fin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2414,7,7.310550,426.0,654.0,Gd,Gd,Av,GLQ,Ex,Gd,Gd,RFn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1989,5,6.875232,480.0,968.0,Gd,TA,No,Unf,TA,TA,No Fireplace,Unf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,625,6,7.239933,514.0,1394.0,TA,Gd,No,BLQ,Gd,TA,Gd,RFn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [39]:
#mapping

exter_qual_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4}
bsmt_qual_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'No Basement':6}
bsmt_exposure_map = {'Gd':1, 'Av':2, 'Mn':3, 'No':4}
bsmtfin_type_1_map = {'GLQ': 1, 'ALQ': 2, 'BLQ': 3, 'Rec': 4, 'LwQ': 5, 'Unf': 6, 'No Basement': 7}
heating_qc_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po': 5}
kitchen_qual_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po': 5}
fireplace_qu_map = {'Ex':1, 'Gd':2, 'TA':3, 'Fa':4, 'Po':5, 'No Fireplace':6}
garage_finish_map = {'Fin':1, 'RFn':2, 'Unf':3, 'No Garage':4}

df_test_model["exter_qual"] = df_test_model["exter_qual"].map(exter_qual_map)
df_test_model["bsmt_qual"] = df_test_model["bsmt_qual"].map(bsmt_qual_map)
df_test_model["bsmt_exposure"] = df_test_model["bsmt_exposure"].map(bsmt_exposure_map)
df_test_model["bsmtfin_type_1"] = df_test_model["bsmtfin_type_1"].map(bsmtfin_type_1_map)
df_test_model["heating_qc"] = df_test_model["heating_qc"].map(heating_qc_map)
df_test_model["kitchen_qual"] = df_test_model["kitchen_qual"].map(kitchen_qual_map)
df_test_model["fireplace_qu"] = df_test_model["fireplace_qu"].map(fireplace_qu_map)
df_test_model["garage_finish"] = df_test_model["garage_finish"].map(garage_finish_map)

#check the OHE + Mapping
df_test_model.head()

,id,overall_qual,log_gr_liv_area,garage_area,total_bsmt_sf,exter_qual,bsmt_qual,bsmt_exposure,bsmtfin_type_1,heating_qc,kitchen_qual,fireplace_qu,garage_finish,neighborhood_Blmngtn,neighborhood_Blueste,neighborhood_BrDale,neighborhood_BrkSide,neighborhood_ClearCr,neighborhood_CollgCr,neighborhood_Crawfor,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Greens,neighborhood_GrnHill,neighborhood_IDOTRR,neighborhood_Landmrk,neighborhood_MeadowV,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NPkVill,neighborhood_NWAmes,neighborhood_NoRidge,neighborhood_NridgHt,neighborhood_OldTown,neighborhood_SWISU,neighborhood_Sawyer,neighborhood_SawyerW,neighborhood_Somerst,neighborhood_StoneBr,neighborhood_Timber,neighborhood_Veenker,exterior_1st_AsbShng,exterior_1st_AsphShn,exterior_1st_BrkComm,exterior_1st_BrkFace,exterior_1st_CBlock,exterior_1st_CemntBd,exterior_1st_HdBoard,exterior_1st_ImStucc,exterior_1st_MetalSd,exterior_1st_Plywood,exterior_1st_Stone,exterior_1st_Stucco,exterior_1st_VinylSd,exterior_1st_Wd Sdng,exterior_1st_WdShing,exterior_2nd_AsbShng,exterior_2nd_AsphShn,exterior_2nd_Brk Cmn,exterior_2nd_BrkFace,exterior_2nd_CBlock,exterior_2nd_CmentBd,exterior_2nd_HdBoard,exterior_2nd_ImStucc,exterior_2nd_MetalSd,exterior_2nd_Plywood,exterior_2nd_Stone,exterior_2nd_Stucco,exterior_2nd_VinylSd,exterior_2nd_Wd Sdng,exterior_2nd_Wd Shng,foundation_BrkTil,foundation_CBlock,foundation_PConc,foundation_Slab,foundation_Stone,foundation_Wood
0,2658,6,7.564238,440.0,1020.0,3,4,4,6,2,4,6,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2718,5,7.584265,580.0,1967.0,3,2,4,6,3,3,6,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2414,7,7.310550,426.0,654.0,2,2,2,1,1,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1989,5,6.875232,480.0,968.0,2,3,4,6,3,3,6,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,625,6,7.239933,514.0,1394.0,3,2,4,3,2,3,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Use Ridge Regression model to predict the saleprice of the houses in test.csv

In [40]:
#scale the features in the train set
ss.fit(X)
X_ss = ss.transform(X)

#model fit based on the X and y variable in the train set
ridgeRegressor.fit(X_ss, y)

#X_test_set is all the entries in test set + scale features
X_test_set = df_test_model.drop(['id'], axis=1)
X_test_set = ss.transform(X_test_set)

#predict the saleprice based on entries in test set
X_test_pred = lassoRegressor.predict(X_test_set)

Save as submission csv file to submit to Kaggle

In [41]:
submission = pd.DataFrame({'ID': df_test_model['id'], 'SalePrice': X_test_pred})
print(submission)

       ID      SalePrice
0    2658  122992.910616
1    2718  184381.485308
2    2414  220791.197056
3    1989  109631.432872
4     625  177839.371432
..    ...            ...
873  1662  179431.332649
874  1234  218738.585090
875  1373  139205.844921
876  1672   96378.798257
877  1939  119222.640386

[878 rows x 2 columns]


In [42]:
submission.to_csv("../Dataset/submission.csv", index = False)

# 8. Conclusion and Recommendations

The aim of this project is to develop a Regression model that can predict to at least 80% accuracy of Iowa property prices by analysing and narrowing down a list of 80 housing features.<br>

In our project, we were given 80 housing features which might affect saleprice to analyse. We have narrrowed it down to 16 features after the EDA process. <br>

We then chose the Ridge Regression model to predict saleprices of houses in Ames as it yields the highest R2 and lowest RMSE value amongst all the regression models. The model has an accuracy of 83% in predicting saleprice. <br>

We believe that with such a model, saleprices of houses in Ames can be predicted more accurately by our housing agents and help our clients to optimise property for sale and maximise the profits of their properties. <br>

Moving forward, our recommendations are:
1. To improve our predictions of housing saleprice by implementing another supervised learning model type to improve the accuracy of prediction to above 90%.
2. The limitations to such a model is that it is specific to Ames, Iowa. Our model can be improved to predict saleprices of houses outside of this area.
3. A more updated dataset (after 2020) can be analysed for a more accurate prediction in saleprice of houses.
4. Other features such as amenities in the area or distance to Iowa State University could be studied as it may impact housing saleprice.